In [1]:
%load_ext jupyter_black

# nexrad

In [2]:
import numpy as np

import boto3
import botocore
from botocore.client import Config
import matplotlib.pyplot as plt
from metpy.io import Level2File
from metpy.plots import add_timestamp, ctables
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [12]:
from datetime import datetime
from typing import NewType, Iterable

import re
import pandas as pd


s3 = boto3.resource(
    "s3",
    config=Config(signature_version=botocore.UNSIGNED, user_agent_extra="Resource"),
)
bucket = s3.Bucket("noaa-nexrad-level2")


class NexradSummary:
    def __init__(self, dsum: pd.Series) -> None:
        self.__date_range_summary = dsum

    def __repr__(self) -> str:
        return self.__date_range_summary.__repr__()

    def itersum(self):
        yield from self.__date_range_summary.items()
    def _generator(self, key:bytes, sweep:int):
        for vt, obj in self.itersum():
            f = Level2File(obj.get()["Body"])
    def reflectivity(self, sweep: int = 0):
        return tuple(self._)

        #     # return f
        #     sweep = 0
        #     # First item in ray is header, which has azimuth angle
        #     az = np.array([ray[0].az_angle for ray in f.sweeps[sweep]])

        #     ref_hdr = f.sweeps[sweep][0][4][b"REF"][0]
        #     ref_range = (
        #         np.arange(ref_hdr.num_gates) * ref_hdr.gate_width + ref_hdr.first_gate
        #     )
        #     self.__ref = np.array([ray[4][b"REF"][1] for ray in f.sweeps[sweep]])

        #     rho_hdr = f.sweeps[sweep][0][4][b"RHO"][0]
        #     rho_range = (
        #         np.arange(rho_hdr.num_gates + 1) - 0.5
        #     ) * rho_hdr.gate_width + rho_hdr.first_gate
        #     rho = np.array([ray[4][b"RHO"][1] for ray in f.sweeps[sweep]])

        #     phi_hdr = f.sweeps[sweep][0][4][b"PHI"][0]
        #     phi_range = (
        #         np.arange(phi_hdr.num_gates + 1) - 0.5
        #     ) * phi_hdr.gate_width + phi_hdr.first_gate
        #     phi = np.array([ray[4][b"PHI"][1] for ray in f.sweeps[sweep]])

        #     zdr_hdr = f.sweeps[sweep][0][4][b"ZDR"][0]
        #     zdr_range = (
        #         np.arange(zdr_hdr.num_gates + 1) - 0.5
        #     ) * zdr_hdr.gate_width + zdr_hdr.first_gate
        #     zdr = np.array([ray[4][b"ZDR"][1] for ray in f.sweeps[sweep]])


def nexrad(date: datetime, station_id: str):
    prefix = date.strftime(f"%Y/%m/%d/{station_id}")

    def generate():
        for obj in bucket.objects.filter(Prefix=prefix):
            key = obj.key.split("/")[-1]
            if not key.endswith("_V06"):
                continue
            d = datetime.strptime(key, f"{station_id}%Y%m%d_%H%M%S_V06")

            if True:
                yield d, obj

    dsum = pd.DataFrame(generate(), columns=["validTime", "summary"]).set_index("validTime")["summary"]

    return NexradSummary(dsum)


lvl2 = nexrad(datetime(2022, 7, 20, 0, 19, 5), "KMLB")
lvl2.reflectivity()

KeyboardInterrupt: 